In [1]:
import os
from ctrace.constraint import load_graph, find_contours
from contact_tracing import PQ
from contact_tracing import *
from ctrace.constraint import ProbMinExposed, ProbMinExposedMIP
os.getcwd()

'/home/zach/Documents/prog/research/ContactTracing/ctrace'

In [2]:
LOADING = False
np.random.seed(42)

In [4]:
# Setup graph and infected sets
if LOADING:
    G = load_graph("mont100")
else:
    G = nx.balanced_tree(4, 3)

# Randomly sample 5% of infected
N = G.number_of_nodes()
num_infected = int(N * 0.05)
I = np.random.choice(N, num_infected, replace=False)

# Find contours
contour1, contour2 = find_contours(G, I)

# Set K value
K = int(0.8 * len(I))

# Find the infected probabilities
p1, q = PQ(G, I)

In [5]:
print(f"K: {K}")
print(f"I: {I}")

K: 3
I: [19 45 72 30]


In [6]:
constraintMIP = ProbMinExposedMIP.from_dataframe(G, I, contour1, contour2, p1, q, K)

p1: 0.5499999999999999
p1: 0.5499999999999999
p1: 0.49999999999999994
p1: 0.65
p1: 0.3
p1: 0.6
p1: 0.35
p1: 0.49999999999999994


In [7]:
constraintMIP.solve_lp()


In [8]:
constraintMIP.getVariables()


array([1., 1., 1., 0., 0., 0., 0., 0.])